In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [ ]:
## Load the trained model , scaled pickle file, one hot and label encoder file
model=load_model('model.h5')

## Load the encoder and scaler
    with open('onehot_encoder.pkl','rb') as file:
        onehot_encoder_geo=pickle.load(file)

    with open('label_encoder.pkl','rb') as file:
        label_encoder_gender=pickle.load(file)

    with  open('scaler.pkl','rb') as file:
        scaler=pickle.load(file)

In [3]:
#Example input data

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
##Onde hot Encode Geography
geo_encoded=onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\Projects\CurnPredictionModel\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_data_df=pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [8]:
##Label encoder
input_data_df['Gender']=label_encoder_gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [9]:
##Concat the onehot data to input data
input_data_df=pd.concat([input_data_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
##Scaled the input data
scaled_data=scaler.transform(input_data_df)
scaled_data

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [11]:
##Prediction
prediction=model.predict(scaled_data)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step


array([[0.02885088]], dtype=float32)

In [12]:
prediction_prob=prediction[0][0]
prediction_prob

np.float32(0.028850878)

In [13]:
if prediction_prob>0.5:
    print("The Customer is Likely to Churn")
else:
    print("The Customer is not likely to Churn")

The Customer is not likely to Churn
